In [1]:
import duckdb


In [11]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [17]:
df = con.execute(""" 
                 SELECT * 
                 FROM (
                        SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao desc) AS ROW
                        FROM bronze_z0019
                     
                       )
                 WHERE ROW = 1

                 """).fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,ROW
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-04-23 00:38:21.662931,1
1,10004,SERRA,BT10,100,200,z0019_2.csv,2025-04-23 00:50:09.909126,1
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-04-23 00:38:21.662931,1
3,10006,PREGO,BT10,100,60,z0019_2.csv,2025-04-23 00:50:09.909126,1
4,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-04-23 00:38:21.662931,1
5,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-04-23 00:50:09.909126,1


In [22]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'ROW'])
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT10,100,200
2,10003,PREGO,BT10,100,50
3,10006,PREGO,BT10,100,60
4,10002,MARTELO,BT50,100,1500
5,10005,MACHADO,BT50,100,100


In [25]:
df2 = df_final
df2 = df2.astype(
    {
        'id': 'int32',
        'nm_produto': 'string',
        'id_categoria': 'string',
        'id_fornecedor': 'int32',
        'vl_preco': 'float32'


    }
)

df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT10,100,200.0
2,10003,PREGO,BT10,100,50.0
3,10006,PREGO,BT10,100,60.0
4,10002,MARTELO,BT50,100,1500.0
5,10005,MACHADO,BT50,100,100.0


In [26]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos(
                id BIGINT,
                nm_produto  TEXT,
                id_categoria	TEXT,
                id_fornecedor BIGINT,
            	vl_preco FLOAT
            
             )
            """)

In [28]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [29]:
df_resultado = con.execute("select * from produtos").fetch_df()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT10,100,200.0
2,10003,PREGO,BT10,100,50.0
3,10006,PREGO,BT10,100,60.0
4,10002,MARTELO,BT50,100,1500.0
5,10005,MACHADO,BT50,100,100.0


In [30]:
con.close()